# 11. Map Bicycle Lanes GSV

Draw maps to compare the bicycle lane routes detected from GSV images in a survey area vs. the OpenStreetMap cycleways and the Principal Bicycle Network

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Which "locality" do we wish to process?
# Assumes that we can find a pair of OSM files with corresponding names,
# extracted with the "osmium" tool.  One file follows the official shape of
# the locality, while a second file follows a bounding box around the locality
# with a 200m margin, so that when we are looking for intersections, we don't
# miss any due to the intersecting road being just outside the boundary of the
# locality (apart from the intersection).
locality = 'Mount Eliza Sample'

# We will sample the middle of each intersection, but we can also sample a
# "margin" around the intersection, at 10m intervals.
# E.g. if we set this to "20" then we will sample points at:
#    -20m, -10m, 0m, 10m, and 20m
# from the centre of the intersection, along the assumed bearing of the road
margin = 20

# Default zoom level for maps
zoom = 14

## Code

In [2]:
# General imports
import json
import os
import sys

from ipyleaflet import Map, Marker, GeoJSON

import pandas as pd

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.detection_map as detection_map

In [3]:
# Derive paths for configuration

# A version of the locality name with spaces replaced by underscores
locality_clean   = locality.replace(' ', '_')

# Name of the locality with the margin around intersections included
locality_margin  = '{0:s}_{1:d}m'.format(locality_clean, margin)

# Detection log with individual points
detection_log = os.path.join(module_path_root, 'detections', locality_margin, 'detection_log.csv')

# Geojson files of detectsions and comparisons to OSM
detected_filename = os.path.join(module_path_root, 'detections', locality_margin, 'hit.geojson')
tag_filename      = os.path.join(module_path_root, 'detections', locality_margin, 'tag.geojson')
both_filename     = os.path.join(module_path_root, 'detections', locality_margin, 'both.geojson')
hit_only_filename = os.path.join(module_path_root, 'detections', locality_margin, 'hit_only.geojson')
tag_only_filename = os.path.join(module_path_root, 'detections', locality_margin, 'tag_only.geojson')

# Geojson file describing the outline of the survey area
margin_filename   = os.path.join(module_path_root, 'data_sources', 'Locality_{0:s}_margin.geojson'.format(locality_clean))

# Geojson file describing the exact outline of the town
LGA_locality = locality.replace(' Sample', '')
locality_filename = os.path.join(os.pardir, 'data_sources', 'LGA_boundary_' + LGA_locality + '.geojson')

## Map 1: Detection Points

Display detections as points on the map, with bounding box for the survey area

In [4]:
# Load boundary of the survey area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

# Find the center of the survey area, so we can center the map
center = detection_map.get_centroid(margin_data)

# Create the lap and add the survey area boundary as a layer
m1 = Map(center=center, zoom=zoom)
m1.add_layer(margin_layer)

# Load detected points and add to map
count = detection_map.load_points(m1, detection_log)
print('Point count: ' + str(count))

# Display the map
m1

Point count: 92


Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Map 2: Detected bicycle lane routes

Infer bicycle lane routes based on the points at which bicycle lane markings were detected by the model

In [5]:
# Load lines that we drew from the detected points
detected_layer, detected_data = detection_map.load_layer(detected_filename, color='red')

# Build the map from the boundary of the survey area, and the detected lanes layer
m2 = Map(center=center, zoom=zoom)
m2.add_layer(margin_layer)
m2.add_layer(detected_layer)

# Display the map
m2

Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Map 3: Cycleways according to OpenStreetMap

We created this map at the same time as we drew the detected bicycle lane routes in map 2, above, and the comparison maps in map 4, below.

In [6]:
# Load lines that we drew from the detected points
tag_layer, tag_data = detection_map.load_layer(tag_filename, color='red')

# Build the map
m3 = Map(center=center, zoom=zoom)
m3.add_layer(margin_layer)
m3.add_layer(tag_layer)

# Display the map
m3

Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Map 4: Comparison of Detection Model and OpenStreetMap

Highlight bicycle lane routes that were either detected by the model, or tagged as cycleways in OpenStreetMap.

* If both sources agree, then the route is drawn in green.
* If the route was only detected by our model, it is drawn in blue.
* If OpenStreetMap has a route taggeed that our model did not detect, it is drawn in red.

In [7]:
# Load lines that we drew from the detected points when we compared to OpenStreetMap
both_layer,     both_data     = detection_map.load_layer(both_filename,     color='green')
hit_only_layer, hit_only_data = detection_map.load_layer(hit_only_filename, color='blue')
tag_only_layer, tag_only_data = detection_map.load_layer(tag_only_filename, color='red')

# Build the map
m4 = Map(center=center, zoom=zoom)
m4.add_layer(margin_layer)
m4.add_layer(both_layer)
m4.add_layer(hit_only_layer)
m4.add_layer(tag_only_layer)

# Display the map
m4

Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## TODO: Comments to put in Results section of thesis

OSM is missing a segment of Nepean Highway from Mount Eliza Way to Old Mornington Road.

This corresponds to way id="204762387".  It is NOT tagged with a "cycleway", it is tagged with "bicycle"="yes" and "foot"="yes".  This generally means that bicycle traffic (and foot traffic) is allowed, but not that there is a cycleway.

The small segment that is missing on Humphries Road from Walkers Road to Overport road was clipped in the OSM extract for the locality, it is just outside the official shape of the locality.  If we widen our OSM extract we can see that it is a cycleway.  way id="671976035".